# edge_condition.py — ELI5 Walkthrough
This notebook gives a super-friendly tour of `python/samples/getting_started/workflows/control-flow/edge_condition.py` from the Agent Framework samples.


## Big Picture
Imagine an office mailroom assistant who checks each email, decides whether it's spam, and either throws it away or drafts a friendly reply for you. The script builds exactly that mailroom as an **Agent Framework workflow** powered by Azure OpenAI agents.

We will walk through the cast of characters, the decision logic, and the workflow wiring so you can see how the pieces click together, no matter your background.


## Key Ingredients
- **Agents & Executors** – Mini workers powered by Azure OpenAI. Executors wrap them so the workflow engine can run them.
- **WorkflowBuilder** – LEGO blocks that connect executors and describe how messages move.
- **Structured Outputs** – Pydantic models (`DetectionResult`, `EmailResponse`) guarantee the AI sends well-formed JSON instead of messy text.
- **Edge Conditions** – Tiny yes/no checks that decide which path the message takes next.
- **Azure Identity** – `AzureCliCredential` reuses your `az login` so you do not hard-code secrets.


### Workflow Diagram
```mermaid
flowchart LR
    Start(["Incoming Email"]) --> Spam[[spam_detection_agent]]
    Spam -- "is_spam == False" --> Transform[[to_email_assistant_request]]
    Transform --> Assistant[[email_assistant_agent]]
    Assistant --> Respond[[handle_email_response]]
    Spam -- "is_spam == True" --> Flag[[handle_spam_classifier_response]]
```


In [ ]:
import asyncio
from typing import Any

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    executor,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
from pydantic import BaseModel
from typing_extensions import Never

load_dotenv()


class DetectionResult(BaseModel):
    """Represents the result of spam detection."""

    is_spam: bool
    reason: str
    email_content: str


class EmailResponse(BaseModel):
    """Represents the response from the email assistant."""

    response: str


def get_condition(expected_result: bool):
    """Create a condition callable that routes based on DetectionResult.is_spam."""

    def condition(message: Any) -> bool:
        if not isinstance(message, AgentExecutorResponse):
            return True
        try:
            detection = DetectionResult.model_validate_json(message.agent_run_response.text)
            return detection.is_spam == expected_result
        except Exception:
            return False

    return condition


@executor(id="send_email")
async def handle_email_response(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    email_response = EmailResponse.model_validate_json(response.agent_run_response.text)
    await ctx.yield_output(f"Email sent:\n{email_response.response}")


@executor(id="handle_spam")
async def handle_spam_classifier_response(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    detection = DetectionResult.model_validate_json(response.agent_run_response.text)
    if detection.is_spam:
        await ctx.yield_output(f"Email marked as spam: {detection.reason}")
    else:
        raise RuntimeError("This executor should only handle spam messages.")


@executor(id="to_email_assistant_request")
async def to_email_assistant_request(
    response: AgentExecutorResponse, ctx: WorkflowContext[AgentExecutorRequest]
) -> None:
    detection = DetectionResult.model_validate_json(response.agent_run_response.text)
    user_msg = ChatMessage(Role.USER, text=detection.email_content)
    await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


def build_workflow(chat_client: AzureOpenAIChatClient):
    spam_detection_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "You are a spam detection assistant that identifies spam emails. "
                "Always return JSON with fields is_spam (bool), reason (string), and email_content (string). "
                "Include the original email content in email_content."
            ),
            response_format=DetectionResult,
        ),
        id="spam_detection_agent",
    )
    email_assistant_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "You are an email assistant that helps users draft professional responses to emails. "
                "Your input may be a JSON object that includes 'email_content'; base your reply on that content. "
                "Return JSON with a single field 'response' containing the drafted reply."
            ),
            response_format=EmailResponse,
        ),
        id="email_assistant_agent",
    )
    builder = (
        WorkflowBuilder()
        .set_start_executor(spam_detection_agent)
        .add_edge(spam_detection_agent, to_email_assistant_request, condition=get_condition(False))
        .add_edge(to_email_assistant_request, email_assistant_agent)
        .add_edge(email_assistant_agent, handle_email_response)
        .add_edge(spam_detection_agent, handle_spam_classifier_response, condition=get_condition(True))
    )
    return builder.build()


async def run_email_workflow(email_text: str):
    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())
    workflow = build_workflow(chat_client)
    request = AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=email_text)],
        should_respond=True,
    )
    events = await workflow.run(request)
    return events.get_outputs()


SAMPLE_EMAILS = {
    "Team follow-up": """Subject: Team Meeting Follow-up - Action Items

Hi Sarah,

I wanted to follow up on our team meeting this morning and share the action items we discussed:

1. Update the project timeline by Friday
2. Schedule client presentation for next week
3. Review the budget allocation for Q4

Please let me know if you have any questions or if I missed anything from our discussion.

Best regards,
Alex Johnson
Project Manager
Tech Solutions Inc.
alex.johnson@techsolutions.com
(555) 123-4567""",
    "Obvious spam": """Subject: 🎉 CONGRATULATIONS! You've WON $1,000,000 - CLAIM NOW! 🎉

Dear Valued Customer,

URGENT NOTICE: You have been selected as our GRAND PRIZE WINNER!

🏆 YOU HAVE WON $1,000,000 USD 🏆

This is NOT a joke! You are one of only 5 lucky winners selected from millions of email addresses worldwide.

To claim your prize, you MUST respond within 24 HOURS or your winnings will be forfeited!

CLICK HERE NOW: http://win-claim.com

What you need to do:
1. Reply with your full name
2. Provide your bank account details
3. Send a processing fee of $500 via wire transfer

ACT FAST! This offer expires TONIGHT at midnight!

Best regards,
Dr. Johnson Williams
International Lottery Commission
Phone: +1-555-999-1234""",
    "Ambiguous": """Subject: Action Required: Verify Your Account

Dear Valued Customer,

We have detected unusual activity on your account and need to verify your identity to ensure your security.

To maintain access to your account, please login to your account and complete the verification process.

Account Details:
- User: johndoe@contoso.com
- Last Login: 08/15/2025
- Location: Seattle, WA
- Device: Mobile

This is an automated security measure. If you believe this email was sent in error, please contact our support team immediately.

Best regards,
Security Team
Customer Service Department""",
}


## The Data ShapesTwo Pydantic models describe the language that agents speak:- `DetectionResult` → `{ "is_spam": bool, "reason": str, "email_content": str }`  - Tells us if the email is spam, why, and includes the original email text for reuse.- `EmailResponse` → `{ "response": str }`  - Holds the drafted reply that the assistant writes when the email is legitimate.With strict schemas, malformed AI output raises an error early instead of silently breaking the workflow.

In [ ]:
import json
from pprint import pprint

sample = {"is_spam": False, "reason": "Looks like a real teammate", "email_content": "Hello there!"}
parsed = DetectionResult.model_validate_json(json.dumps(sample))
pprint(parsed.model_dump())


The snippet above mimics a spam detector response. `model_validate_json` keeps the workflow honest by rejecting anything that does not match `DetectionResult`.

## The Decision Gate (`get_condition`)`get_condition(expected_result)` builds a tiny function that checks the last agent response:1. If the message is not an `AgentExecutorResponse`, let it pass (to avoid dead ends).2. Otherwise, parse the JSON into `DetectionResult`.3. Compare `is_spam` with the expected value.   - `True` → follow the spam route.   - `False` → follow the helpful reply route.4. Parsing failures return `False`, so the workflow errs on the side of *not* taking that edge.This is how the workflow makes a left or right turn without peppering the rest of the code with `if` statements.

In [ ]:
import json
from types import SimpleNamespace

payload = {"is_spam": True, "reason": "Suspicious giveaway", "email_content": "You won!"}
mock_response = AgentExecutorResponse(
    executor_id="spam_detection_agent",
    agent_run_response=SimpleNamespace(text=json.dumps(payload)),
)
spam_edge = get_condition(True)
not_spam_edge = get_condition(False)
spam_edge(mock_response), not_spam_edge(mock_response)


## The Executors (Little Workers)- `spam_detection_agent` – Reads the email and returns a `DetectionResult` JSON blob.- `to_email_assistant_request` – Converts that JSON into a new user message for the assistant.- `email_assistant_agent` – Drafts a polite reply and returns an `EmailResponse` JSON object.- `handle_email_response` – Prints the drafted reply as the workflow output.- `handle_spam_classifier_response` – If marked spam, prints a short notice instead.Each executor is tiny and single-purpose, so the control flow stays easy to follow.

In [ ]:
import asyncio
import json
from types import SimpleNamespace

payload = {"is_spam": False, "reason": "Legit teammate", "email_content": "Hi Sarah"}
mock_response = AgentExecutorResponse(
    executor_id="spam_detection_agent",
    agent_run_response=SimpleNamespace(text=json.dumps(payload)),
)
captured_requests = []

async def capture(message):
    captured_requests.append(message)

mock_ctx = SimpleNamespace(send_message=capture)

async def demo():
    await to_email_assistant_request(mock_response, mock_ctx)

await demo()
captured_requests[0]


## Wiring the WorkflowHere is the decision tree in Python data form. Only one branch (spam vs. not spam) fires for each email.

In [ ]:
[
    ("spam_detection_agent", "to_email_assistant_request", "is_spam == False"),
    ("to_email_assistant_request", "email_assistant_agent", "always"),
    ("email_assistant_agent", "handle_email_response", "always"),
    ("spam_detection_agent", "handle_spam_classifier_response", "is_spam == True"),
]


## What `main()` Does
1. Logs in to Azure OpenAI using your CLI credentials.
2. Creates agents with the instructions and structured response formats shown above.
3. Builds the workflow graph with conditional edges for spam vs. non-spam routes.
4. Pulls a sample email from `SAMPLE_EMAILS` so the run stays self-contained.
5. Wraps the email in an `AgentExecutorRequest` and runs the workflow.
6. Prints either the drafted reply or a spam warning.
Because everything is asynchronous, `asyncio.run(main())` in the script kicks off the process when you run it as a module.


## Try It Out With Sample Emails
The loop below iterates over the inline `SAMPLE_EMAILS` dictionary and runs the workflow for each one.
This uses the real agents defined in the script, so make sure your Azure OpenAI environment is configured.


In [ ]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    # Jupyter/VS Code notebooks already have an event loop, so await directly.
    await main()
else:
    asyncio.run(main())


## Takeaways- **Structured outputs** keep AI responses predictable and make conditional routing safe.- **Edge conditions** are reusable predicates that keep the workflow diagram tidy.- **Executors** act like small adapters that isolate responsibilities (detect spam, translate, respond, print outcome).- You can extend the pattern with new branches ("urgent", "needs manager", etc.) by adding more predicates and executors.Treat Agent Framework workflows like flowcharts of reliable little workers passing structured messages along the line.